In [13]:
from pyspark.sql import SparkSession, types, functions as F

In [8]:
spark = SparkSession.builder\
          .master('local[*]')\
          .appName('test')\
          .getOrCreate()

In [ ]:
measurements_schema = types.StructType([
  types.StructField('day', types.DateType(), False),
  types.StructField('interval', types.IntegerType(), False),
  types.StructField('detid', types.StringType(), False),
  types.StructField('flow', types.IntegerType(), True),
  types.StructField('occ', types.FloatType(), True),
  types.StructField('error', types.FloatType(), True),
  types.StructField('city', types.StringType(), False),
  types.StructField('speed', types.FloatType(), True)
])

In [20]:
df = spark.read\
      .option('header', True)\
      .schema(measurements_schema)\
      .csv('./data/measurements_test.csv')

In [21]:
df.printSchema()

root
 |-- day: date (nullable = true)
 |-- interval: integer (nullable = true)
 |-- detid: string (nullable = true)
 |-- flow: integer (nullable = true)
 |-- occ: float (nullable = true)
 |-- error: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- speed: float (nullable = true)



In [22]:
df.show(20)

+----------+--------+--------+----+----+-----+--------+-----+
|       day|interval|   detid|flow| occ|error|    city|speed|
+----------+--------+--------+----+----+-----+--------+-----+
|2017-05-06|       0|06.X-2li|  12| 0.0| NULL|augsburg| NULL|
|2017-05-06|     300|06.X-2li|  12| 0.0| NULL|augsburg| NULL|
|2017-05-06|     600|06.X-2li|  12| 0.0| NULL|augsburg| NULL|
|2017-05-06|     900|06.X-2li|  16| 0.0| NULL|augsburg| NULL|
|2017-05-06|    1200|06.X-2li|  16| 0.0| NULL|augsburg| NULL|
|2017-05-06|    1500|06.X-2li|  16| 0.0| NULL|augsburg| NULL|
|2017-05-06|    1800|06.X-2li|  20| 0.0| NULL|augsburg| NULL|
|2017-05-06|    2100|06.X-2li|  20| 0.0| NULL|augsburg| NULL|
|2017-05-06|    2400|06.X-2li|  20| 0.0| NULL|augsburg| NULL|
|2017-05-06|    2700|06.X-2li|   8|0.01| NULL|augsburg| NULL|
|2017-05-06|    3000|06.X-2li|   8|0.01| NULL|augsburg| NULL|
|2017-05-06|    3300|06.X-2li|   8|0.01| NULL|augsburg| NULL|
|2017-05-06|    3600|06.X-2li|   4| 0.0| NULL|augsburg| NULL|
|2017-05

In [7]:
spark.stop()